# Dict and sets 

These are all based on hash tables.

## About sets

In [73]:
# a quick way to create sets using *
args1 = [1,2,3]
args2 = (5,6,7)
s = {*args1, *args2, 8, 8, 8, 10}
print(s)

# sets can be updated inplace with union...
s |= {22}
print(s)

# or removing 
s -= {22}
print(s)

# or intersections 
s &= {1,2,3,4,99}
print(s)

# or symmetric difference (i.e. only keeping elements that are not in both sets)
s ^= {2,4}
print(s) 

{1, 2, 3, 5, 6, 7, 8, 10}
{1, 2, 3, 5, 6, 7, 8, 10, 22}
{1, 2, 3, 5, 6, 7, 8, 10}
{1, 2, 3}
{1, 3, 4}


## swt vs frozen sets

There are also frozen sets. Also these do not preserve the order. They can not be modified.

In [82]:
s = {2,3}
f = frozenset({2,3,4})

# frozen sets don;t have add/remove
s.add(1)
print(s)

# but we can do normal operations on them, including inplace
f -= s
print(f)

{1, 2, 3}
frozenset({4})


## Empty dictionaries and dict_keys as sorted sets

In [55]:
# 1. Preallocating dictionaries may save memory, as we don't need updatin ghte dict hash table.
dict.fromkeys(['a','b','c'])

# 2. This is also a way to keep a unique list of items (like a set), but preserving their order (which set can not do)
d = dict.fromkeys(['a','b','c','b','a'])
print(d.keys())

# 3. finally, dict_keys objects a dynamic, and will update as I add/remove keys
keys = d.keys()
d['new'] = 3 
print(keys)

# finally: dict_keys, dict_items, dict_values have a lot of methods similar to sets.
new_keys = {'a': 1, 1:1, 2:2}.keys()
keys & new_keys

dict_keys(['a', 'b', 'c'])
dict_keys(['a', 'b', 'c', 'new'])


{'a'}

## Merging Mappings

In [3]:
# | overator
print( {'a': 0} | {'b': 1} )
# or 
print({**{'a': 0}, **{'b': 1}, 'other':'stuff'}) 

{'a': 0, 'b': 1}
{'a': 0, 'b': 1, 'other': 'stuff'}


In [4]:
# You can also "add incrementally"
a = {'a': 0} 
a |= {'b': 1}
print(a)

{'a': 0, 'b': 1}


## `setdefault`

This is a mapping method that avoids unnecessary key lookups. The usage is more clearly explained in the example below.

In [14]:
# For each word, let's create a list of positions when the word is found in the text
some_text = 'bike car home car car home'
d = {} 
for ii,w in enumerate(some_text.split(' ')):
    # very ugly code. 
    positions = d.get(w, []) # get list - default to []
    positions.append(ii)     # add new positions
    d[w] = positions         # update key
print(d)

d = {} 
for ii,w in enumerate(some_text.split(' ')):
    # compact code:
    d.setdefault(w,[]).append(ii)
print(d)

# the setdefault function does two things at the same time, exemplified in the code below. 
# 1. it adds a default to the key w, if not found.
# 2. returns the object itself.
d = {} 
for ii,w in enumerate(some_text.split(' ')):
    if w not in d:
        d[w] = []
    d[w].append(ii)
print(d)

{'bike': [0], 'car': [1, 3, 4], 'home': [2, 5]}
{'bike': [0], 'car': [1, 3, 4], 'home': [2, 5]}
{'bike': [0], 'car': [1, 3, 4], 'home': [2, 5]}


## Automatic handling of missing keys

The `get` and `setdefault` methods are not the only possibility. If you want the handling of missing values even when using the `[key]` notation you can either use the `collections.defaultdict` or subclassing `dict` adding a `__missing__` method to it.

In [27]:
# collections.defaultdict will automatically default to something. This is a callable, though. 
# So if I want it to default to an empty list, I pass `list`, or `lambda: []`, but not `[]`.
# We can also specify extra arg and kwargs to be called.
import collections
some_text = 'bike car home car car home'
d = collections.defaultdict(list)
for ii,w in enumerate(some_text.split(' ')):
    d[w].append(ii)
print(d)

defaultdict(<class 'list'>, {'bike': [0], 'car': [1, 3, 4], 'home': [2, 5]})


In [36]:
# IMPORTANT: In general, it is better to sunclass from `collections.UserDict`; e.g. in this scenario,
# we'd avoid having to write the `self.get` method. 

class MyDict(dict):
    """A dictionary subclass than behaves like collections.defaultdict(list)."""
    def __missing__(self, key):
        """Assign an empty list to missing keys, and return their value"""
        self[key] = []   # here we assign default empty list
        return self[key] # here we return it.
    def get(self, key):
        '''self.__missing__ is only called by __getitem__ (hence self.[]), not by `self.get`. Hence,
        without this, `self.get(missing_key)` will raise an error.
        We would not need this method if we were subclassing from `collections.UserDict`'''
        return self[key]
d = MyDict()
for ii,w in enumerate(some_text.split(' ')):
    d[w].append(ii)
print(d)
print(d.get('missing_key'))
print(d)


{'bike': [0], 'car': [1, 3, 4], 'home': [2, 5]}
[]
{'bike': [0], 'car': [1, 3, 4], 'home': [2, 5], 'missing_key': []}


## Custom dictionaries from `UserDict`
A major difference than subclassing from `dict` directly is that data will be stored in an actual dictionary under `self.data`. This avoids messing up (e.g. getting infinite recursion) if we are to modify the special methods.

Below we build a dictionary that encrypts its keys.

In [44]:
from typing import Any 
from collections import UserDict
# from cryptography.fernet import Fernet

class EncryptedDict(UserDict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # key = Fernet.generate_key()
        # self._fernet = Fernet(key)

    def encrypt_key(self, key: str):
        """For simplicity, we simply hash the key. But we left the code required for proper
        encryption with Fernet"""
        # return self._fernet.encrypt(key.encode())
        return hash(key)
    
    def __setitem__(self, key: str, item: Any) -> None:
        key_encrypted = self.encrypt_key(key)
        # self[key] = item # this also works
        return super().__setitem__(key_encrypted, item)
    
    def __getitem__(self, key: Any) -> Any:
        key_encrypted = self.encrypt_key(key)
        return self.data[key_encrypted]
        # return super().__getitem__(key_encrypted)

ed = EncryptedDict()
ed['cacca'] = 32
ed['merdina'] = 64
print(ed)
print(ed['cacca'])
    


{5286198421476792899: 32, -3538111422514561573: 64}
32


## ChainMap
This is a list of mappings. When you query a key, it first looks it into the first mapping, then the second, etc until the key is found.

In [39]:
from collections import ChainMap
c = ChainMap(
    {'a': 1, 'b': 2}, 
    {'a': 11, 'b': 22, 'c': 3}
)
print(c['a'])
print(c['c'])

1
3


## Immutable Mapping

There are no immutable mappings. But we can have some ways around. For example the 

In [49]:
from types import MappingProxyType
d_backend = {'a': 1, 'b': 2}
d_front = MappingProxyType(d_backend)
print(d_front)

# assignments will fail...
# d_front['a'] = 3

# but changes in the backend dictionary will be reflected
d_backend['c'] = 3
print(d_front)

{'a': 1, 'b': 2}
{'a': 1, 'b': 2, 'c': 3}
